In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

In [4]:
mpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml'

model = load_model(mpath,'libsbml')

In [5]:
# RUN AGAIN - some thing about the explode does not work (reac part)
kpf = KEGGapFiller('sha')
gene_table = kpf.get_missing_genes(model)
reac_table = kpf.get_missing_reacs(model, gene_table)


100%|██████████| 1987/1987 [34:40<00:00,  1.05s/it]


In [12]:
reac_table

,ec-code,ncbiprotein,id,equation,reference,is_transport,via
0,1.1.1.100,[BAE04992],MNXR111352,1 MNXM5@MNXD1 + 1 MNXM735182@MNXD1 = 1 MNXM725...,keggR:R07763,None,MetaNetX
1,1.1.1.100,[BAE04992],MNXR116437,1 MNXM10017@MNXD1 + 1 MNXM5@MNXD1 = 1 MNXM1009...,seedR:rxn21009,None,MetaNetX
2,1.1.1.100,[BAE04992],MNXR116441,1 MNXM10018@MNXD1 + 1 MNXM5@MNXD1 = 1 MNXM1009...,seedR:rxn21013,None,MetaNetX
3,1.1.1.100,[BAE04992],MNXR117851,1 MNXM5@MNXD1 + 1 MNXM97406@MNXD1 = 1 MNXM7387...,seedR:rxn23310,None,MetaNetX
4,1.1.1.100,[BAE04992],MNXR118085,1 MNXM137@MNXD1 + 1 MNXM97393@MNXD1 = 1 MNXM16...,seedR:rxn28224,None,MetaNetX
...,...,...,...,...,...,...,...
3611,2.3.1.313,[BAE04147],R13186,None,{'brenda': ['2.3.1.313']},None,KEGG
3612,2.4.99.28,"[BAE04395, BAE04771]",R06178,None,"{'kegg.pathway': ['ec00550'], 'brenda': ['2.4....",None,KEGG
3613,2.4.99.28,"[BAE04395, BAE04771]",R06179,None,"{'kegg.pathway': ['ec00550'], 'brenda': ['2.4....",None,KEGG
3614,2.7.10.3,[BAE03698],R02584,None,{'brenda': ['2.7.10.3']},None,KEGG


In [27]:
gene_table

,orgid:locus,locus_tag,kegg.orthology,ec-code,ncbiprotein,uniprot
0,sha:pSHaeA01,pSHaeA01,[K11210],[2.5.1.-],[BAE05988],[Q4L2Y9]
1,sha:pSHaeA02,pSHaeA02,NaN,None,[BAE05989],[Q4L2Y8]
2,sha:pSHaeA03,pSHaeA03,NaN,None,[BAE05990],[Q4L2Y7]
3,sha:pSHaeB01,pSHaeB01,[K00561],[2.1.1.184],[BAE05991],[Q4L2Y6]
4,sha:pSHaeB02,pSHaeB02,NaN,None,[BAE05992],[Q4L2Y5]
...,...,...,...,...,...,...
1982,sha:SH2674,SH2674,[K03501],[2.1.1.170],[BAE05983],[Q4L2Z4]
1983,sha:SH2675,SH2675,[K03495],None,[BAE05984],[Q4L2Z3]
1984,sha:SH2676,SH2676,[K03650],[3.6.-.-],[BAE05985],[Q4L2Z2]
1985,sha:SH2677,SH2677,[K03536],[3.1.26.5],[BAE05986],[Q4L2Z1]
